## 산업별 AI 혁신 사례

---

### 데이터 확인하고 모델 학습하기

In [ ]:
# main.py
import machine as ma

def main():
    
    """
    지시사항 1번. 데이터를 읽고 처리하는 코드를 작성해보세요.
    """
    
    x_train_us, x_test_us, y_train_us, y_test_us = ma.preprocess()
    
    
    """
    지시사항 2번. 학습을 수행시켜보세요.
    """
    model = ma.train(x_train_us, y_train_us)
    
    
if __name__ == "__main__":
    main()


In [ ]:
# machine.py
import numpy as np 
import pandas as pd 
import sys
from tqdm import trange

import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')

# to avoid warnings
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, precision_score, recall_score, average_precision_score

from elice_utils import EliceUtils
elice_utils = EliceUtils()

def preprocess():

    print("데이터 읽는 중...")
    data = pd.read_csv('data/uci-secom.csv')
    
    print("읽어 온 데이터를 출력합니다.")
    print(data)
    
    #print(data.isnull().sum())
    #print("결측 데이터 값을 처리합니다.")
    data = data.replace(np.NaN, 0)
    #print(data.isnull().sum())
    
    # 쓸모없는 데이터 지우기
    data = data.drop(columns = ['Time'], axis = 1)
    
    # 타겟 데이터 분리
    x = data.iloc[:,:590]
    y = data.iloc[:, 590]

    print("\n학습용 데이터와 테스트용 데이터로 분리 합니다.")

    # Under sampling 수행
    failed_tests = np.array(data[data['Pass/Fail'] == 1].index)
    no_failed_tests = len(failed_tests)

    normal_indices = data[data['Pass/Fail'] == -1]
    no_normal_indices = len(normal_indices)
    
    np.random.seed(10)
    random_normal_indices = np.random.choice(no_normal_indices, size = no_failed_tests, replace = True)
    random_normal_indices = np.array(random_normal_indices)

    under_sample = np.concatenate([failed_tests, random_normal_indices])
    undersample_data = data.iloc[under_sample, :]

    x = undersample_data.iloc[:, undersample_data.columns != 'Pass/Fail'] 
    y = undersample_data.iloc[:, undersample_data.columns == 'Pass/Fail']
    y = np.ravel(y)

    x_train_us, x_test_us, y_train_us, y_test_us = train_test_split(x, y, test_size = 0.2, random_state = 4)
    
    print("학습용 데이터 크기: {}".format(x_train_us.shape))
    print("테스트용 데이터 크기: {}".format(x_test_us.shape))
    
    return x_train_us, x_test_us, y_train_us, y_test_us

def train(x_train_us, y_train_us):

    print("학습을 수행합니다.")
    
    model = XGBClassifier(random_state=2)
    
    # 파라미터 튜닝
    parameters = [{'max_depth' : [1, 2, 3, 4, 5, 6]}]
    
    grid_search = GridSearchCV(estimator = model, param_grid = parameters, scoring = 'recall', cv = 4, n_jobs = -1)
    
    
    import pickle
    
    for j in trange(5000,file=sys.stdout, leave=False, unit_scale=True, desc='학습 진행률'):
    
        with open('model.pkl', 'rb') as f:
             model = pickle.load(f)
    print("학습 완료")
    
    return model


---

### 학습 모델 평가하기

In [ ]:
# main.py
import machine as ma

def main():
    
    """
    지시사항 1번. 예측 정확도 결과를 출력해보세요.
    """
    ma.evaluation()

if __name__ == "__main__":
    main()


In [ ]:
# machine.py
import numpy as np 
import pandas as pd 


import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')

# to avoid warnings
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, precision_score, recall_score, average_precision_score

from elice_utils import EliceUtils
elice_utils = EliceUtils()

def preprocess():

    #print("데이터 읽는 중...")
    data = pd.read_csv('data/uci-secom.csv')
    
    #print(data.isnull().sum())
    #print("결측 데이터 값을 처리합니다.")
    data = data.replace(np.NaN, 0)
    #print(data.isnull().sum())
    
    # 쓸모없는 데이터 지우기
    data = data.drop(columns = ['Time'], axis = 1)
    
    # 타겟 데이터 분리
    x = data.iloc[:,:590]
    y = data.iloc[:, 590]

    #print("학습용 데이터와 테스트용 데이터로 분리 합니다.")

    # Under sampling 수행
    failed_tests = np.array(data[data['Pass/Fail'] == 1].index)
    no_failed_tests = len(failed_tests)

    normal_indices = data[data['Pass/Fail'] == -1]
    no_normal_indices = len(normal_indices)
    
    np.random.seed(10)
    random_normal_indices = np.random.choice(no_normal_indices, size = no_failed_tests, replace = True)
    random_normal_indices = np.array(random_normal_indices)

    under_sample = np.concatenate([failed_tests, random_normal_indices])
    undersample_data = data.iloc[under_sample, :]

    x = undersample_data.iloc[:, undersample_data.columns != 'Pass/Fail'] 
    y = undersample_data.iloc[:, undersample_data.columns == 'Pass/Fail']
    y = np.ravel(y)

    x_train_us, x_test_us, y_train_us, y_test_us = train_test_split(x, y, test_size = 0.2, random_state = 4)
    
    #print("학습용 데이터 크기: {}".format(x_train_us.shape))
    #print("테스트용 데이터 크기: {}".format(x_test_us.shape))
    
    return x_train_us, x_test_us, y_train_us, y_test_us
    
    
def train(x_train_us, y_train_us):

    """
    #print("학습을 수행합니다.")
    
    model = XGBClassifier(random_state=2)
    
    # 파라미터 튜닝
    parameters = [{'max_depth' : [1, 2, 3, 4, 5, 6]}]

    grid_search = GridSearchCV(estimator = model, param_grid = parameters, scoring = 'recall', cv = 4, n_jobs = -1)

    grid_search = grid_search.fit(x_train_us, y_train_us)
    
    # 베스트 모델 선택
    model = grid_search.best_estimator_
    """
    import pickle
    
    with open('model.pkl', 'rb') as f:
         model = pickle.load(f)
    
    return model
    
def evaluation():

    x_train_us, x_test_us, y_train_us, y_test_us = preprocess()
    model = train(x_train_us, y_train_us)
    # 테스트 데이터 예측
    y_pred = model.predict(x_test_us)
    print('평가 지표인 recall score를 출력합니다.: ', recall_score(y_test_us, y_pred), '\n')
    
    print("센서들의 중요도를 출력합니다.")
    xgb.plot_importance(model, height = 1, grid = True, importance_type = 'gain', show_values = False, max_num_features = 20)

    plt.rcParams['figure.figsize'] = (10, 15)
    plt.xlabel('The importance score for each features')
    plt.ylabel('Features')
    
    plt.savefig("result1.png")
    elice_utils.send_image("result1.png")

    


---

### 공정 이상 예측하기

In [ ]:
# main.py
import machine as ma

def main():
	
    """
    지시사항 1번. 103번 센서값인 아래의 value_103_sensor 값을 바꾸어보세요.
    """
    value_103_sensor = -40
    
    # 예측을 진행하는 코드입니다.
    ma.predict(value_103_sensor)
    
if __name__ == "__main__":
    main()


In [ ]:
# machine.py
import numpy as np 
import pandas as pd 


import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')

# to avoid warnings
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, precision_score, recall_score, average_precision_score

from elice_utils import EliceUtils
elice_utils = EliceUtils()

def preprocess():

    #print("데이터 읽는 중...")
    data = pd.read_csv('data/uci-secom.csv')
    
    #print(data.isnull().sum())
    #print("결측 데이터 값을 처리합니다.")
    data = data.replace(np.NaN, 0)
    #print(data.isnull().sum())
    
    # 쓸모없는 데이터 지우기
    data = data.drop(columns = ['Time'], axis = 1)
    
    # 타겟 데이터 분리
    x = data.iloc[:,:590]
    y = data.iloc[:, 590]

    #print("학습용 데이터와 테스트용 데이터로 분리 합니다.")

    # Under sampling 수행
    failed_tests = np.array(data[data['Pass/Fail'] == 1].index)
    no_failed_tests = len(failed_tests)

    normal_indices = data[data['Pass/Fail'] == -1]
    no_normal_indices = len(normal_indices)
    
    np.random.seed(10)
    random_normal_indices = np.random.choice(no_normal_indices, size = no_failed_tests, replace = True)
    random_normal_indices = np.array(random_normal_indices)

    under_sample = np.concatenate([failed_tests, random_normal_indices])
    undersample_data = data.iloc[under_sample, :]

    x = undersample_data.iloc[:, undersample_data.columns != 'Pass/Fail'] 
    y = undersample_data.iloc[:, undersample_data.columns == 'Pass/Fail']
    y = np.ravel(y)

    x_train_us, x_test_us, y_train_us, y_test_us = train_test_split(x, y, test_size = 0.2, random_state = 4)
    
    #print("학습용 데이터 크기: {}".format(x_train_us.shape))
    #print("테스트용 데이터 크기: {}".format(x_test_us.shape))
    
    return x_train_us, x_test_us, y_train_us, y_test_us, data
    
    
def train(x_train_us, y_train_us):

    """
    #print("학습을 수행합니다.")
    
    model = XGBClassifier(random_state=2)
    
    # 파라미터 튜닝
    parameters = [{'max_depth' : [1, 2, 3, 4, 5, 6]}]

    grid_search = GridSearchCV(estimator = model, param_grid = parameters, scoring = 'recall', cv = 4, n_jobs = -1)

    grid_search = grid_search.fit(x_train_us, y_train_us)
    
    # 베스트 모델 선택
    model = grid_search.best_estimator_
    """
    import pickle
    
    with open('model.pkl', 'rb') as f:
         model = pickle.load(f)
    
    return model
    
def predict(value_103_sensor):

    x_train_us, x_test_us, y_train_us, y_test_us, data = preprocess()
    model = train(x_train_us, y_train_us)
    
    pre_data = data
    pre_data.loc[1242, '103'] = value_103_sensor
    pre_data = pre_data[1242:1243]
    
    prediction = model.predict(pre_data.drop(columns = 'Pass/Fail'))
    
    
    fig, ax = plt.subplots(figsize=(8, 6))
    
    sns.distplot(data['103'], color = 'darkblue')
    plt.title('103 Sensor Measurements', fontsize = 20)
    
    ax.annotate('103_sensor_value',
            xy=(value_103_sensor, 0), xycoords='data',
            xytext=(10, 30), textcoords='offset points',
            arrowprops=dict(facecolor='black', shrink=0.05),
            horizontalalignment='left', verticalalignment='bottom')
    
    plt.savefig("result1.png")
    elice_utils.send_image("result1.png")
    
    if prediction == 1:
    
        print("103 센서 데이터 값이 {}인 경우 공정 이상이 발생할 것으로 예측됩니다.".format(value_103_sensor))
    else:
        print("103 센서 데이터 값이 {}인 경우 공정 이상이 발생하지 않을 것으로 예측됩니다.".format(value_103_sensor))
    
    

---
---




## 이커머스 산업의 AI

### 데이터 확인

In [ ]:
# main.py

import machine as ma

def main():
    
    """
    지시사항 1번. 인공지능 모델 학습을 수행해보세요.
    """
    netflix_overall, cosine_sim, indices = ma.preprocess()


if __name__ == "__main__":
    main()


In [ ]:
# machine.py
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

import sys
from tqdm import trange

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def clean_data(x):
    return str.lower(x.replace(" ", ""))

def create_soup(x):
    return x['title']+ ' ' + x['director'] + ' ' + x['cast'] + ' ' +x['listed_in']+' '+ x['description']

def preprocess():

    pd.set_option('display.max_columns', None)
    print("데이터 읽는 중...")
    netflix_overall=pd.read_csv("data/netflix_titles.csv")

    print("읽어 온 데이터를 출력합니다.")
    print(netflix_overall)
    
    filledna=netflix_overall.fillna('')

    features=['title','director','cast','listed_in','description']
    filledna=filledna[features]

    for feature in features:
        filledna[feature] = filledna[feature].apply(clean_data)

    filledna['soup'] = filledna.apply(create_soup, axis=1)
    
    count = CountVectorizer(stop_words='english')
    count_matrix = count.fit_transform(filledna['soup'])

    cosine_sim = cosine_similarity(count_matrix, count_matrix)

    filledna=filledna.reset_index()
    indices = pd.Series(filledna.index, index=filledna['title'])
    
    print("학습을 수행합니다.")
    
    for j in trange(20,file=sys.stdout, leave=False, unit_scale=True, desc='학습 진행률'):
        
        cosine_sim = cosine_similarity(count_matrix, count_matrix)
    
    print('학습이 완료되었습니다.')
    return netflix_overall, cosine_sim, indices

---


### 추천 알고리즘 형태

In [ ]:
#main.py
import machine as ma

def main():
    
    netflix_overall, cosine_sim, indices = ma.preprocess()
    
    """
    지시사항 1번. 따옴표 사이에 들어가 있는 영화명을 지우고 왼쪽 지문의 예시 중 원하는 영화명을 입력해보세요.
    """
    title = 'Pororo - The Little Penguin'
    
    print("{}와 비슷한 넷플릭스 콘텐츠를 추천합니다.".format(title))
    ma.get_recommendations_new(title, netflix_overall, cosine_sim, indices)


if __name__ == "__main__":
    main()


In [ ]:
# machine.py
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def clean_data(x):
    return str.lower(x.replace(" ", ""))

def create_soup(x):
    return x['title']+ ' ' + x['director'] + ' ' + x['cast'] + ' ' +x['listed_in']+' '+ x['description']

def preprocess():

    netflix_overall=pd.read_csv("data/netflix_titles.csv")

    filledna=netflix_overall.fillna('')

    features=['title','director','cast','listed_in','description']
    filledna=filledna[features]

    for feature in features:
        filledna[feature] = filledna[feature].apply(clean_data)

    filledna['soup'] = filledna.apply(create_soup, axis=1)
    
    count = CountVectorizer(stop_words='english')
    count_matrix = count.fit_transform(filledna['soup'])

    cosine_sim = cosine_similarity(count_matrix, count_matrix)

    filledna=filledna.reset_index()
    indices = pd.Series(filledna.index, index=filledna['title'])
    
    return netflix_overall, cosine_sim, indices
    
    
def get_recommendations_new(title, netflix_overall, cosine_sim, indices):
    
    pd.set_option('display.max_columns', None)
    title=title.replace(' ','').lower()
    
    try:
        idx = indices[title]

        sim_scores = list(enumerate(cosine_sim[idx]))

        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

        sim_scores = sim_scores[1:11]

        movie_indices = [i[0] for i in sim_scores]

        recomendation = netflix_overall[['title','country','release_year']].iloc[movie_indices]
        
        sim_pd = pd.DataFrame(sim_scores)[[1]]
        
        sim_pd = sim_pd.rename(columns={1:'Similiarity'})

        recomendation = recomendation.reset_index(drop=True)

        recomendation = pd.concat([recomendation, sim_pd], axis=1)
    
        recomendation.index += 1
        
        
        return print(recomendation)
    
    except:
        print("오류: 올바른 title 명을 적어주세요.")

---


## 웹/앱 서비스 산업 AI

### 이미지 분할

In [ ]:
# main.py
import machine as ma

def main():
    
    """
    지시사항 1번. 손글씨 이미지에서 숫자 부분만 분리하는 과정을 수행해보세요.
    """
    ma.data_print()

if __name__ == "__main__":
    main()


In [ ]:
# machine.py
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
import numpy as np

import cv2
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

from elice_utils import EliceUtils
elice_utils = EliceUtils()

    
    
def data_print():
    
    img = cv2.imread("data/numbers.jpg")
    
    print("원본 이미지를 출력합니다.")
    plt.figure(figsize=(15,12))
    plt.imshow(img);
    
    plt.savefig("result2.png")
    elice_utils.send_image("result2.png")
    
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img_blur = cv2.GaussianBlur(img_gray, (5, 5), 0)
    img_th = cv2.threshold(img_blur, 155, 250, cv2.THRESH_BINARY_INV)[1]
    
    contours, hierachy= cv2.findContours(img_th, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

    rects = [cv2.boundingRect(each) for each in contours]
    
    tmp = [w*h for (x,y,w,h) in rects]
    tmp.sort()
    
    rects = [(x,y,w,h) for (x,y,w,h) in rects if ((w*h>1000)and(w*h<500000))]
    
    print("\n이미지를 분할할 영역을 표시합니다.")
    for rect in rects:
    # Draw the rectangles
        cv2.rectangle(img, (rect[0], rect[1]), (rect[0] + rect[2], rect[1] + rect[3]), (0, 255, 0), 5) 
    plt.clf()
    plt.figure(figsize=(15,12))
    plt.imshow(img);
    plt.savefig("result3.png")
    elice_utils.send_image("result3.png")
    
    seg_img = []

    margin_pixel = 50

    for cnt in contours: 
        x, y, w, h = cv2.boundingRect(cnt) 

        # Drawing a rectangle on copied image 
        if ((w*h>1000)and(w*h<500000)): 

            # Cropping the text block for giving input to OCR 
            cropped = img.copy()[y - margin_pixel:y + h + margin_pixel, x - margin_pixel:x + w + margin_pixel] 
            seg_img.append(cropped)

            rect = cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 255), 5)

    print("\n분할 된 이미지를 출력합니다.")
    for i in range(len(seg_img)):
        plt.clf()
        plt.imshow(seg_img[i]);
        plt.savefig("result4.png")
        elice_utils.send_image("result4.png")
    
    
    re_seg_img = []

    for i in range(len(seg_img)):
        re_seg_img.append(cv2.resize(seg_img[i], (28,28), interpolation=cv2.INTER_AREA))
    
    gray = cv2.cvtColor(re_seg_img[0], cv2.COLOR_BGR2GRAY)
    


---

## 손글씨 분류

In [ ]:
# main.py
import machine as ma

def main():
    
    """
    지시사항 1번. MNIST 데이터를 학습하는 인공지능 모델을 구현해보세요.
    """
    model = ma.train()

if __name__ == "__main__":
    main()


In [ ]:
# machine.py
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
import numpy as np

import cv2
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

from elice_utils import EliceUtils
elice_utils = EliceUtils()

def train():
    mnist = tf.keras.datasets.mnist

    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    x_train, x_test = x_train / 255.0, x_test / 255.0

    # model에 batch normalization layer 추가
    model = tf.keras.models.Sequential([
      tf.keras.layers.Flatten(input_shape=(28, 28)),
      tf.keras.layers.Dense(128, activation='relu'),
      tf.keras.layers.Dense(128, activation='relu'),
      tf.keras.layers.Dense(10, activation='softmax')
      #tf.keras.layers.Dense(10, activation='relu')
    ])

    # adam외의 optimizer로 변경
    # sparse_categorical_crossentropy외의 loss로 변경
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    # epochs 값 변경
    model.fit(x_train, y_train, epochs=5)

    model.evaluate(x_test,  y_test, verbose=2)

    # 임의의 5가지 test data의 이미지와 레이블값을 출력하고 예측된 레이블값 출력
    predictions = model.predict(x_test)
    idx_n = [15, 34, 68, 75, 98]

    for i in idx_n:

        img = x_test[i].reshape(28,28)
        plt.imshow(img,cmap="gray")
        plt.show()
        plt.savefig("result1.png")
        elice_utils.send_image("result1.png")

        print("Label: ", y_test[i])
        print("Prediction: ", np.argmax(predictions[i]))
        
    return model
    
    


---

### 예측

In [ ]:
# main.py

import machine as ma

def main():
    
    """
    지시사항 1번. 이미지 변환 과정과 예측 과정을 수행해보세요.
    """
    ma.data_predit()

if __name__ == "__main__":
    main()


In [ ]:
# machine.py
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
import numpy as np

import cv2
import matplotlib.pyplot as plt

import warnings
warnings.simplefilter("ignore")

from elice_utils import EliceUtils
elice_utils = EliceUtils()

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

from tensorflow.python.util import deprecation
deprecation._PRINT_DEPRECATION_WARNINGS = False

tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
    
def data_predit():

    # model에 batch normalization layer 추가
    model = tf.keras.models.Sequential([
      tf.keras.layers.Flatten(input_shape=(28, 28)),
      tf.keras.layers.Dense(128, activation='relu'),
      tf.keras.layers.Dense(128, activation='relu'),
      tf.keras.layers.Dense(10, activation='softmax')
      #tf.keras.layers.Dense(10, activation='relu')
    ])

    # adam외의 optimizer로 변경
    # sparse_categorical_crossentropy외의 loss로 변경
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    
    import os

    checkpoint_path = "./cp.ckpt"
    checkpoint_dir = os.path.dirname(checkpoint_path)
    
    model.load_weights(checkpoint_path)


    
    img = cv2.imread("data/numbers.jpg")
    
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img_blur = cv2.GaussianBlur(img_gray, (5, 5), 0)
    img_th = cv2.threshold(img_blur, 150, 250, cv2.THRESH_BINARY_INV)[1]
    
    contours, hierachy= cv2.findContours(img_th, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

    rects = [cv2.boundingRect(each) for each in contours]
    
    tmp = [w*h for (x,y,w,h) in rects]
    tmp.sort()
    
    rects = [(x,y,w,h) for (x,y,w,h) in rects if ((w*h>1000)and(w*h<500000))]
    
    for rect in rects:
    # Draw the rectangles
        cv2.rectangle(img, (rect[0], rect[1]), (rect[0] + rect[2], rect[1] + rect[3]), (0, 255, 0), 5) 
    
    
    seg_img = []

    margin_pixel = 50

    for cnt in contours: 
        x, y, w, h = cv2.boundingRect(cnt) 

        # Drawing a rectangle on copied image 
        if ((w*h>1500)and(w*h<600000)): 

            # Cropping the text block for giving input to OCR 
            cropped = img.copy()[y - margin_pixel:y + h + margin_pixel, x - margin_pixel:x + w + margin_pixel] 
            seg_img.append(cropped)

            rect = cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 255), 5)

    re_seg_img = []

    for i in range(len(seg_img)):
        re_seg_img.append(cv2.resize(seg_img[i], (28,28), interpolation=cv2.INTER_AREA))
    
    gray = cv2.cvtColor(re_seg_img[0], cv2.COLOR_BGR2GRAY)
    
    for i in range(len(seg_img)):
    
        gray = cv2.cvtColor(re_seg_img[i], cv2.COLOR_BGR2GRAY)
        img_binary = cv2.threshold(gray, 150, 250, cv2.THRESH_BINARY_INV)[1]
        test = img_binary.reshape(1,28,28) / 255.0

        #print(len(test))

        predictions = model.predict(test)

        img_test = test.reshape(28,28)
        plt.clf()
        
        plt.subplot(121)
        plt.imshow(seg_img[i])
        plt.title('Origin')

        plt.subplot(122)
        plt.imshow(img_test,cmap="gray")
        plt.title('Coverted')
        plt.show()
        
        plt.savefig("result4.png")
        elice_utils.send_image("result4.png")

        #print("Label: ", y_test[i])
        print("Prediction: ", np.argmax(predictions))

---


## 금융 산업 AI

### 데이터 확인

In [ ]:
# main.py
import machine as ma

def main():
    
    """
    지시사항 1번. 출력된 데이터를 확인하고 학습을 수행해보세요.
    """
    ma.data_plot()
    

if __name__ == "__main__":
    main()


In [ ]:
# draw.py
# -*- coding: utf-8 -*-

import base64 as _base64
import fcntl as _fcntl
import mimetypes as _mimetypes
import os.path as _ospath
import struct as _struct

import numpy as np
import pandas as pd
import matplotlib as mpl
mpl.use("Agg")
import matplotlib.pyplot as plt
import sklearn.decomposition
import sklearn.preprocessing
import sklearn.cluster

from elice_utils import EliceUtils  # isort:skip

elice_utils = EliceUtils()

def display_digits(X, title):
    plt.clf()
    plt.figure(figsize=(4, 4))
    plt.imshow(X, cmap=plt.cm.gray_r, interpolation='nearest')
    plt.title(title)

    plt.savefig("image.svg", format="svg")
    elice_utils.send_image("image.svg")


In [ ]:
# machine.py
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from tensorflow.keras import models
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, Dense, Activation
from elice_utils import EliceUtils
elice_utils = EliceUtils()

def data_plot():

    # --- 주식 데이터 로드, 전처리, 분할, 모델 학습하기(이전 실습에서 진행) --- #
    df = pd.read_csv('data/stock.csv') 

    # 데이터프레임 출력(데이터프레임은 (헹 X 열)로 이루어진 표 형태의 특수한 데이터 타입)
    print(df)


    # --- 주식 데이터 살펴보기 --- #

    print('\n주식 데이터의 형태를 출력')
    print(df.shape)

    print('\n주식 데이터의 정보를 출력')
    print(df.info)

    print('\n주식 데이터의 상단 5개 행을 출력')
    print(df.head())

    print('\n주식 데이터의 하단 5개 행을 출력')
    print(df.tail())

    print('\n주식 데이터의 모든 열을 출력')
    print(df.columns)

    print('\n주식 데이터의 요약 통계 자료 출력')
    print(df.describe())

def train():

    # --- 주식 데이터 로드, 전처리, 분할, 모델 학습하기(이전 실습에서 진행) --- #
    df = pd.read_csv('data/stock.csv') 

    # 주가의 중간값 계산하기
    high_prices = df['High'].values
    low_prices = df['Low'].values
    mid_prices = (high_prices + low_prices) / 2

    # 주가 데이터에 중간 값 요소 추가하기
    df['Mid'] = mid_prices

    # 종가의 5일 이동평균값을 계산하고 주가 데이터에 추가하기
    ma5 = df['Adj Close'].rolling(window=5).mean()
    df['MA5'] = ma5

    df = df.fillna(0) # 비어있는 값을 모두 0으로 바꾸기

    # Date 열를 제거합니다.
    df = df.drop('Date', axis = 1)

    # 데이터 스케일링(MinMaxScaler 적용)
    min_max_scaler = MinMaxScaler()
    fitted = min_max_scaler.fit(df)

    output = min_max_scaler.transform(df)
    output = pd.DataFrame(output, columns=df.columns, index=list(df.index.values))

    # 트레인셋/테스트셋 크기 설정
    train_size = int(len(output)* 0.6) # 트레인셋은 전체의 60%
    test_size = int(len(output)*0.3) + train_size # 테스트셋은 전체의 30%

    #train/test 학습 및 라벨 설정
    #종가를 예측하기 위해 종가를 label로 설정
    train_x = np.array(output[:train_size])
    train_y = np.array(output['Close'][:train_size])
    test_x =np.array(output[train_size:test_size])
    test_y = np.array(output['Close'][train_size:test_size])
    validation_x = np.array(output[test_size:])
    validation_y = np.array(output['Close'][test_size:])

    # Keras 모델을 생성합니다.
    model = Sequential()

    # Keras 딥러닝 모델 학습을 위한 파라미터(옵션값)을 설정합니다.
    learning_rate = 0.01
    training_cnt = 1000
    batch_size = 100 
    input_size = 8 

    # 생성된 딥러닝 모델에 학습용 데이터(train_x)를 넣습니다.
    model.add(Dense(input_size, activation='tanh', input_shape=(train_x.shape[1],))) 
    model.add(Dense(input_size * 3,  activation='tanh')) 
    model.add(Dense(1, activation='tanh'))

    # 데이터를 학습을 진행합니다.
    model.compile(optimizer='sgd', loss='mse', metrics=['mae', 'mape','acc'])
    model.summary()
    history = model.fit(train_x, train_y, epochs=training_cnt,   
                        batch_size=batch_size, verbose=1)
    val_mse, val_mae, val_mape, val_acc = model.evaluate(test_x, test_y, verbose=0)
    
    
def predict():

    # --- 학습 결과를 그래프로 확인해봅니다 --- #

    # 학습된 모델로부터 테스트 데이터를 예측합니다.
    pred = model.predict(test_x)

    fig = plt.figure(facecolor='white', figsize=(8, 5))
    ax = fig.add_subplot(111)
    ax.plot(test_y, label='True') # 실제 주가
    ax.plot(pred, label='Prediction') # 우리가 만든 딥러닝 모델이 예측한 주가
    ax.legend()

    # 현재까지 그려진 그래프를 시각화
    plt.savefig("plot.png")
    elice_utils.send_image("plot.png")

---


### 파생 변수 생성

In [ ]:
# main.py
import machine as ma

def main():
    
    """
    지시사항 1번. 이동평균값(MA),거래량 이동평균값(VMA), 이격도값(disp) 변수를 추가해보세요.
    """
    ma.data_preprocess()

if __name__ == "__main__":
    main()


In [ ]:
# machine.py
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from tensorflow.keras import models
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, Dense, Activation
from elice_utils import EliceUtils
elice_utils = EliceUtils()

def data_plot():

    # --- 주식 데이터 로드, 전처리, 분할, 모델 학습하기(이전 실습에서 진행) --- #
    df = pd.read_csv('data/stock.csv') 

    # 데이터프레임 출력(데이터프레임은 (헹 X 열)로 이루어진 표 형태의 특수한 데이터 타입)
    print(df)


    # --- 주식 데이터 살펴보기 --- #

    print('\n주식 데이터의 형태를 출력')
    print(df.shape)

    print('\n주식 데이터의 정보를 출력')
    print(df.info)

    print('\n주식 데이터의 상단 5개 행을 출력')
    print(df.head())

    print('\n주식 데이터의 하단 5개 행을 출력')
    print(df.tail())

    print('\n주식 데이터의 모든 열을 출력')
    print(df.columns)

    print('\n주식 데이터의 요약 통계 자료 출력')
    print(df.describe())
    
def data_preprocess():

    # 주식 데이터 불러오기
    df = pd.read_csv('data/stock.csv') 


    # 수정 종가 이동평균(MA: Moving Average) 값 구하기
    ma5 = df['Adj Close'].rolling(window=5).mean() # 수정 종가 5일 이동평균
    ma20 = df['Adj Close'].rolling(window=20).mean() # 수정 종가 20일 이동평균
    ma60 = df['Adj Close'].rolling(window=60).mean() # 수정 종가 60일 이동평균


    # 이동평균 값 추가하기
    df.insert(len(df.columns), "MA5", ma5) # 'MA5'라는 열 이름으로 ma5 값 추가
    df.insert(len(df.columns), "MA20", ma20) # 'MA20'라는 열 이름으로 ma20 값 추가
    df.insert(len(df.columns), "MA60", ma60) # 'MA60'라는 열 이름으로 ma60 값 추가


    # 거래량 5일 이동평균 추가
    vma5 = df['Volume'].rolling(window=5).mean() # 거래량의 5일 이동평균 구하기
    df.insert(len(df.columns), "VMA5", vma5) # 'VMA5'라는 열 이름으로 vma5 값 추가


    # --- 이격도 추가 --- #
    # 수정 종가 데이터를 5일 이동평균 값으로 나눈 비율
    disp5 = (df['Adj Close']/df['MA5'])*100

    # 이격도 데이터를 'Disp5'라는 열 이름으로 추가
    df.insert(len(df.columns), "Disp5", disp5) 



    # 데이터 확인
    print('이동평균 및 이격도가 추가된 주가 데이터')
    print(df)
    
    
    
    # 이동평균선 시각화
    print("\n이동평균선 그래프")
    plt.plot(df['Adj Close'], label="Adj Close") # 수정 종가
    plt.plot(df['MA5'], label="MA5") # 종가 5일 이동평균
    plt.plot(df['MA20'], label="MA20") # 종가 20일 이동평균
    plt.plot(df['MA60'], label="MA60") # 종가 60일 이동평균

    # 시각화 옵션 코드
    # (시각화 강의에서 별도로 다루는 내용입니다)
    plt.legend(loc='best')
    plt.xticks(rotation = 45)
    plt.grid()
    plt.savefig("plot2.png")
    elice_utils.send_image("plot2.png")
    
    
    plt.cla()
    # 거래량 이동평균값 시각화하기
    print("\n거래량 이동평균값 그래프")
    plt.plot(df.index, df['VMA5'], label='VMA5') # 거래량 5일 이동평균
    plt.plot(df.index, df['Volume'], label='Volume') # 거래량 데이터

    # 시각화 옵션 코드
    # (시각화 강의에서 별도로 다루는 내용입니다)
    plt.legend(loc='best')
    plt.xticks(rotation = 45)
    plt.grid()
    plt.savefig("plot.png")
    elice_utils.send_image("plot.png")

def train():

    # --- 주식 데이터 로드, 전처리, 분할, 모델 학습하기(이전 실습에서 진행) --- #
    df = pd.read_csv('data/stock.csv') 

    # 주가의 중간값 계산하기
    high_prices = df['High'].values
    low_prices = df['Low'].values
    mid_prices = (high_prices + low_prices) / 2

    # 주가 데이터에 중간 값 요소 추가하기
    df['Mid'] = mid_prices

    # 종가의 5일 이동평균값을 계산하고 주가 데이터에 추가하기
    ma5 = df['Adj Close'].rolling(window=5).mean()
    df['MA5'] = ma5

    df = df.fillna(0) # 비어있는 값을 모두 0으로 바꾸기

    # Date 열를 제거합니다.
    df = df.drop('Date', axis = 1)

    # 데이터 스케일링(MinMaxScaler 적용)
    min_max_scaler = MinMaxScaler()
    fitted = min_max_scaler.fit(df)

    output = min_max_scaler.transform(df)
    output = pd.DataFrame(output, columns=df.columns, index=list(df.index.values))

    # 트레인셋/테스트셋 크기 설정
    train_size = int(len(output)* 0.6) # 트레인셋은 전체의 60%
    test_size = int(len(output)*0.3) + train_size # 테스트셋은 전체의 30%

    #train/test 학습 및 라벨 설정
    #종가를 예측하기 위해 종가를 label로 설정
    train_x = np.array(output[:train_size])
    train_y = np.array(output['Close'][:train_size])
    test_x =np.array(output[train_size:test_size])
    test_y = np.array(output['Close'][train_size:test_size])
    validation_x = np.array(output[test_size:])
    validation_y = np.array(output['Close'][test_size:])

    # Keras 모델을 생성합니다.
    model = Sequential()

    # Keras 딥러닝 모델 학습을 위한 파라미터(옵션값)을 설정합니다.
    learning_rate = 0.01
    training_cnt = 1000
    batch_size = 100 
    input_size = 8 

    # 생성된 딥러닝 모델에 학습용 데이터(train_x)를 넣습니다.
    model.add(Dense(input_size, activation='tanh', input_shape=(train_x.shape[1],))) 
    model.add(Dense(input_size * 3,  activation='tanh')) 
    model.add(Dense(1, activation='tanh'))

    # 데이터를 학습을 진행합니다.
    model.compile(optimizer='sgd', loss='mse', metrics=['mae', 'mape','acc'])
    model.summary()
    history = model.fit(train_x, train_y, epochs=training_cnt,   
                        batch_size=batch_size, verbose=1)
    val_mse, val_mae, val_mape, val_acc = model.evaluate(test_x, test_y, verbose=0)
    
    
def predict():

    # --- 학습 결과를 그래프로 확인해봅니다 --- #

    # 학습된 모델로부터 테스트 데이터를 예측합니다.
    pred = model.predict(test_x)

    fig = plt.figure(facecolor='white', figsize=(8, 5))
    ax = fig.add_subplot(111)
    ax.plot(test_y, label='True') # 실제 주가
    ax.plot(pred, label='Prediction') # 우리가 만든 딥러닝 모델이 예측한 주가
    ax.legend()

    # 현재까지 그려진 그래프를 시각화
    plt.savefig("plot.png")
    elice_utils.send_image("plot.png")

---


### 주가 예측

In [ ]:
# main.py
import machine as ma

def main():
    
    """
    지시사항 1번. 딥러닝 모델의 학습과 예측을 수행해보세요.
    """
    ma.train()

if __name__ == "__main__":
    main()


In [ ]:
# machine.py
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from elice_utils import EliceUtils
from tensorflow.keras.layers import Dense, LSTM, ReLU
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, LSTM, ReLU
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.losses import MSE
elice_utils = EliceUtils()

def rev_min_max_func(scaled_val):

    df = pd.read_csv('data/stock.csv') 
    df = df.fillna(0)
    max_val = max(df['Close'])
    min_val = min(df['Close'])
    og_val = (scaled_val * (max_val - min_val)) + min_val
    return og_val

def data_plot():

    # --- 주식 데이터 로드, 전처리, 분할, 모델 학습하기(이전 실습에서 진행) --- #
    df = pd.read_csv('data/stock.csv') 

    # 데이터프레임 출력(데이터프레임은 (헹 X 열)로 이루어진 표 형태의 특수한 데이터 타입)
    print(df)


    # --- 주식 데이터 살펴보기 --- #

    print('\n주식 데이터의 형태를 출력')
    print(df.shape)

    print('\n주식 데이터의 정보를 출력')
    print(df.info)

    print('\n주식 데이터의 상단 5개 행을 출력')
    print(df.head())

    print('\n주식 데이터의 하단 5개 행을 출력')
    print(df.tail())

    print('\n주식 데이터의 모든 열을 출력')
    print(df.columns)

    print('\n주식 데이터의 요약 통계 자료 출력')
    print(df.describe())

def train():

    # Keras 딥러닝 모델 학습을 위한 파라미터(옵션값)을 설정합니다.
    learning_rate = 0.01
    epochs = 1000
    batch_size = 200
    input_size = 5
    time_step = 1 

    # --- 주식 데이터 로드, 전처리, 분할, 모델 학습하기(이전 실습에서 진행) --- #
    df = pd.read_csv('data/stock.csv') 

    # 주가의 중간값 계산하기
    high_prices = df['High'].values
    low_prices = df['Low'].values
    mid_prices = (high_prices + low_prices) / 2

    # 주가 데이터에 중간 값 요소 추가하기
    df['Mid'] = mid_prices

    # 종가의 5일 이동평균값을 계산하고 주가 데이터에 추가하기
    ma5 = df['Adj Close'].rolling(window=5).mean()
    df['MA5'] = ma5

    df = df.fillna(0) # 비어있는 값을 모두 0으로 바꾸기

    # Date 열를 제거합니다.
    df = df.drop('Date', axis = 1)

    # 데이터 스케일링(MinMaxScaler 적용)
    min_max_scaler = MinMaxScaler()
    fitted = min_max_scaler.fit(df)

    output = min_max_scaler.transform(df)
    output = pd.DataFrame(output, columns=df.columns, index=list(df.index.values))

    x_data = output[['Open', 'Low', 'High', 'Mid', 'MA5']]
    y_data = output['Close']

    train_size = int(len(x_data) * 0.7)
    test_size = int(len(y_data) * 0.3) + train_size

    train_x = np.array(x_data[:train_size])
    train_x = train_x.reshape((train_x.shape[0], time_step, input_size))
    train_y = np.array(y_data[:train_size])

    """
    validation_x = np.array(x_data[train_size:validation_size])
    validation_x = validation_x.reshape((validation_x.shape[0], time_step, input_size))
    validation_y = np.array(y_data[train_size:validation_size])
    """
    test_x = np.array(x_data[train_size:])
    test_x = test_x.reshape((test_x.shape[0], time_step, input_size))
    test_y = np.array(y_data[train_size:])

    # Keras 모델을 생성합니다.
    model = Sequential()


    # 생성된 딥러닝 모델에 학습용 데이터(train_x)를 넣습니다.
    model.add(LSTM(512, input_shape=(time_step, input_size)))
    model.add(Dense(1))
    model.add(ReLU())
    model.compile(optimizer=RMSprop(), loss=MSE, metrics=['mae', 'mape'])

    # 데이터를 학습을 진행합니다.
    model.summary()
    history = model.fit(train_x, train_y, epochs=epochs,   
                        batch_size=batch_size, verbose=1)
    model.evaluate(test_x, test_y, verbose=0)
    

    # --- 학습 결과를 그래프로 확인해봅니다 --- #

    # 학습된 모델로부터 테스트 데이터를 예측합니다.
    pred = model.predict(test_x)

    fig = plt.figure(facecolor='white', figsize=(8, 5))
    ax = fig.add_subplot(111)
    ax.plot(rev_min_max_func(test_y), label='True') # 실제 주가
    ax.plot(rev_min_max_func(pred), label='Prediction') # 우리가 만든 딥러닝 모델이 예측한 주가
    ax.legend()

    # 현재까지 그려진 그래프를 시각화
    plt.savefig("plot.png")
    elice_utils.send_image("plot.png")